In [1]:
import sys
sys.path.append('../')

from EM import NiftiManager, Evaluate, EM, ElastixTransformix, FileManager
from glob import glob
import os
from tqdm import tqdm

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [2]:
NM    = NiftiManager()
EVAL  = Evaluate()
ET    = ElastixTransformix()
FM    = FileManager()

Loading the Registration parameters

In [3]:
os.listdir('./registration-parameters')

['Par0010affine.txt', 'Par0010bspline.txt']

In [4]:
reg_params = '-p "./registration-parameters/Par0010affine.txt" -p "./registration-parameters/Par0010bspline.txt"'

Loading the dataset

In [5]:
train_path = '../TrainingValidationTestSets/Training_Set'
valid_path = '../TrainingValidationTestSets/Validation_Set'
test_path  = '../TrainingValidationTestSets/Test_Set'

In [6]:
train_volumes = sorted(glob(os.path.join(train_path, "***", "IBSR_*.nii.gz"), recursive=True))
train_volumes = [path for path in train_volumes if 'seg' not in path]

train_labels  = sorted(glob(os.path.join(train_path, "***", "IBSR_*_seg.nii.gz"), recursive=True))

In [7]:
valid_volumes = sorted(glob(os.path.join(valid_path, "***", "IBSR_*.nii.gz"), recursive=True))
valid_volumes = [path for path in valid_volumes if 'seg' not in path]

valid_labels  = sorted(glob(os.path.join(valid_path, "***", "IBSR_*_seg.nii.gz"), recursive=True))

In [8]:
valid_volumes

['../TrainingValidationTestSets/Validation_Set\\IBSR_11\\IBSR_11.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_12\\IBSR_12.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_13\\IBSR_13.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_14\\IBSR_14.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_17\\IBSR_17.nii.gz']

In [9]:
test_volumes = sorted(glob(os.path.join(test_path, "***", "IBSR_*.nii.gz"), recursive=True))
test_volumes = [path for path in test_volumes if 'seg' not in path]

Registration and label propagation

In [31]:
# we will assume the valid set is the test to be able to evaluate our approach for now

for fixed_volume, fixed_label in zip(valid_volumes, valid_labels): 
    image_id = fixed_volume.replace('\\', '/').split("/")[-1].split(".")[0]

    print(f"------------- Fixed {image_id} -------------")
    
    for train_volume, train_label in tqdm(zip(train_volumes, train_labels)):
        reg_moving_name = train_volume.replace("\\", "/").split("/")[-1].split(".")[0]

        # register an example train volume to the test space
        ET.register_elastix(
            fixed_path = fixed_volume, 
            moving_path = train_volume,
            reg_params = reg_params,
            create_dir_callback = FM.create_directory_if_not_exists,
            excute_cmd_callback = ET.excute_cmd)
    
        # perform label propagation to test space
        ET.label_propagation_transformix(
            fixed_path = fixed_volume, 
            moving_path = train_label, 
            input_label = train_label,
            transform_path = f'output/images/output_{image_id}/{reg_moving_name}/TransformParameters.1.txt',
            replace_text_in_file_callback = FM.replace_text_in_file,
            create_dir_callback = FM.create_directory_if_not_exists,
            excute_cmd_callback = ET.excute_cmd)
    

------------- Fixed IBSR_11 -------------


10it [24:29, 146.91s/it]


------------- Fixed IBSR_12 -------------


10it [24:24, 146.42s/it]


------------- Fixed IBSR_13 -------------


10it [41:36, 249.68s/it]


------------- Fixed IBSR_14 -------------


10it [24:21, 146.19s/it]


------------- Fixed IBSR_17 -------------


10it [24:59, 149.90s/it]
